# Genetic algorithms: 1D example

Import some packages/modules

In [ ]:
import numpy as np
import pandas as pd
from random import random
import plotly.express as px
import plotly.graph_objects as go
from time import sleep
from IPython.display import clear_output
from rich.table import Table
from rich.theme import Theme
from rich.console import Console
from rich.panel import Panel
from icecream import ic

from fonctions import * # TODO remove by moving content in the notebook

# Set the seed of the random number generator.
# `np.random.seed(value)` is considered a legacy function,
# so let's use a `np.random.Generator`
rng: np.random.Generator = np.random.default_rng(seed=112358)

# create a Rich Console object
# TODO not a global var, pass it to function
console = Console(theme=Theme({"repr.number": ""})) # no special style for numbers, affecting chromosome printing

Notebook settings

In [ ]:
EXPORT_FIGURES = True

# Objective function

Define the function to minimize : $f(x) = −0.02x \times sin(0.01 x \times 2 \pi) − 4$

With $x$ an integer between $0$ and $2^8 = 255$ included.

Within the framework of genetic algorithms, we seek to maximize the fitness score of individuals, regarding their environment &rarr; the score is therefore $-f(x)$.

$f(x)$ is defined so that the score is always positive.

In [ ]:
function_to_minimize = lambda x: -0.02 * x * np.sin(0.01*x*2*np.pi) - 4

fitness_score = lambda x: -function_to_minimize(x)

# compute its value over the domain
x = np.arange(0, 2**8) # range [0, 2^8=256[ -> [0, 255]
y = function_to_minimize(x) # evaluate all values in x

# define how to plot the function
def plot_objective_function(x: np.ndarray, y: np.ndarray) -> go.Figure:
    fig = go.Figure(
        go.Scatter(x=x,y=y,mode='lines',name='objective function'),
        layout_xaxis_range=[0,255],
        layout_yaxis_range=[-9,0]
    )
    fig.update_layout(title_text="Function to minimize")
    return fig

# plot the function
fig = plot_objective_function(x,y)
fig.show()

if EXPORT_FIGURES:
    fig.write_image("function_to_minimize.png")

print(f"The minimum is {np.min(y):0.2f} at x={np.argmin(y)}")

# Binary representation

In order to use a genetic algorithm, we must be able to represent solutions (= individuals) as chromosomes -> alleles vector.

The simplest way to do so with integers is to use their binary representation.

But there is a catch: to enforce adjacent inputs to have adjacent binary codes, we must use the [Gray code](https://en.wikipedia.org/wiki/Gray_code).

This way, the binary representation of two successive integers will differ in only one bit.

We need to functions, `dec2gc()` to compute the Gray code of a decimal number, and `gc2dec()` the inverse.

In [ ]:
# https://www.geeksforgeeks.org/decimal-equivalent-gray-code-inverse/

def dec2gc(dec,N):
    """
    Decimal to Gray code conversion
    """
    binary = dec
    binary ^= (binary >> 1) # conversion happens here
    # convert to string with bin(), remove '0b', pad with '0's for fixed width
    binary = '{:0>{width}}'.format(bin(binary)[2:], 'b', width=N)
    # separate chars with ' ' and use this separator to get an int array
    return np.fromstring(" ".join(binary),dtype=int,sep=" ")

def gc2dec(gc):
    """
    Gray code to decimal conversion
    """
    #create a string from the array
    gc = np.array2string(gc, separator='')[1:-1]
    gc = int(gc,base=2)
    inv = 0
    while(gc):
        inv = inv ^ gc
        gc = gc >> 1
    return inv

Test the conversion functions

In [ ]:

decimal_value = np.random.randint(0,2**8) # random 8 bits integer
gray_code = dec2gc(decimal_value,8)
print(f"The Gray code (on 8 bits) of {decimal_value} is\n{gray_code}\n")
back_to_decimal = gc2dec(gray_code)
print(f"The decimal value of\n{gray_code} is {back_to_decimal}")
assert(back_to_decimal == decimal_value)

Compact string representation of chromosomes

In [ ]:
chromosome2str = lambda chromosome: ''.join([str(allele) for allele in chromosome]) # `chromosome` being a numpy array of int (row of `population` defined below)

chromosome2str(gray_code)

# Generation of the initial population

The population is stored as a $N ~ \text{individuals} \times 8 ~ \text{genes}$ matrix. Here $N=20$.

Option A : pre-generated random individuals

In [ ]:
population = np.array([[1,1,0,1,0,0,0,1],
                       [1,0,0,0,1,1,1,1],
                       [0,1,1,1,1,0,0,0],
                       [1,1,1,1,1,1,1,1],
                       [1,1,0,1,1,0,1,0],
                       [0,1,0,1,0,1,1,0],
                       [0,1,0,1,1,0,0,0],
                       [1,0,1,0,1,1,0,0],
                       [1,1,1,0,0,1,0,0],
                       [1,0,1,0,1,1,1,0],
                       [0,1,0,1,0,0,0,0],
                       [1,0,1,0,0,0,1,0],
                       [1,0,1,1,1,0,0,1],
                       [0,0,0,0,1,1,0,1],
                       [1,0,0,1,1,0,0,0],
                       [0,1,0,0,0,1,0,0],
                       [0,1,1,0,1,1,1,0],
                       [1,0,0,0,0,1,1,1],
                       [1,1,1,1,0,0,1,0],
                       [1,0,0,0,0,0,0,0]], dtype=int)

Option B : populate the initial population with random individuals

In [ ]:
population = rng.integers(low=0, high=2, size=(20,8))

Evaluate each individual of the initial population

In [ ]:
# define how to evaluate a given population
def evaluate_population(population: np.ndarray) -> np.ndarray:
    scores = np.zeros((population.shape[0],1)) # a column vector of size N, N = number of individuals
    for idx in range(0,population.shape[0]):
        scores[idx] = fitness_score(gc2dec(population[idx,:]))
    return scores

# evaluate the initial population
scores: np.ndarray = evaluate_population(population)

# define how to display the scores
def display_scores(population: np.ndarray, scores: np.ndarray, console: Console):
    assert(population.shape[0] == scores.shape[0])
    table = Table(title="Fitness scores")
    table.add_column('Index')
    table.add_column('Chromosome')
    table.add_column('Score')
    for idx in range(0,population.shape[0]):
        table.add_row(
            str(idx),
            chromosome2str(population[idx,:]),
            f'{scores[idx,0]:0.2f}'
        )
    console.print(table)

def compute_stats(values: np.ndarray) -> tuple[float,float,float]:
    return values.mean(), values.max(), values.std()

def print_generation_stats(mean: float, max: float, std: float):
    console.print(
        Panel.fit(
            f'mean score = {mean:0.2f}\n' +
            f'best score = {max:0.2f}\n' +
            f' std. dev. = {std:0.2f}',
            title=f'Score stats'
        )
    )

# display the scores of the initial population
display_scores(population,scores,console)
mean, max, std = compute_stats(scores)
print_generation_stats(mean,max,std)

Plot the population

In [ ]:
# define how to plot a given population
def plot_population(x,y,population) -> go.Figure:
    fig = plot_objective_function(x,y)
    x_population = np.apply_along_axis((lambda x: float(gc2dec(x))), axis=1, arr=population)
    y_population = function_to_minimize(x_population)
    fig.add_trace(go.Scatter(
        x=x_population,
        y=y_population,
        mode='markers',
        marker_color='black',
        marker_size=10,
        name='individuals'
    ))
    fig.layout.update(showlegend=False) # remove legend
    return fig

# plot the initial population
fig = plot_population(x,y,population)
fig.update_layout(title_text='Initial population')
fig.show()

if EXPORT_FIGURES:
    fig.write_image("initial_population.png")

# Crossover

In pairs, the chromosomes of two individuals are recombined according to a crossover point

In [ ]:
def individuals_crossover(parent1: np.ndarray, parent2: np.ndarray, crossover_point: int) -> tuple[np.ndarray, np.ndarray]:
    assert(parent1.shape == (8,))
    assert(parent2.shape == (8,))
    # chromosomes of 8 genes
    # |0|1|2|3|4|5|6|7|
    #   ^ ^ ^ ^ ^ ^ ^
    #   0 1 2 3 4 5 6
    # -> 7 crossover points possible
    assert(crossover_point >= 0)
    assert(crossover_point <= 6)
    child1 = np.copy(parent1)
    child2 = np.copy(parent2)
    temp = np.copy(child1[0:crossover_point+1]) # copy the first chunk of child1
    child1[0:crossover_point+1] = np.copy(child2[0:crossover_point+1]) # replace the first chunk of child1 by the first chunk of child2
    child2[0:crossover_point+1] = np.copy(temp) # replace the first chunk of child2 by the saved first chunk of child1
    return child1, child2

# define how to visualize a crossover between two chromosomes

def display_crossover(parent1: np.ndarray, parent2: np.ndarray, crossover_point: int, child1: np.ndarray, child2: np.ndarray):
    console.print(
        f'parent1 : [bright_red]{chromosome2str(parent1)}[/]\n'
        f'parent2 : [bright_cyan]{chromosome2str(parent2)}[/]\n'
        f' child1 : [bright_cyan]{chromosome2str(child1)[:crossover_point+1]}[/][bright_red]{chromosome2str(child1)[crossover_point+1:]}[/]\n'
        f' child2 : [bright_red]{chromosome2str(child2)[:crossover_point+1]}[/][bright_cyan]{chromosome2str(child2)[crossover_point+1:]}[/]'
    )

# With predefined parents and crossover point

parent1 = np.array([1,1,0,1,0,0,0,1], dtype=int)
parent2 = np.array([0,0,0,0,1,1,0,1], dtype=int)
crossover_point = 3
child1, child2 = individuals_crossover(parent1,parent2,crossover_point)
display_crossover(
    parent1,
    parent2,
    crossover_point,
    child1,
    child2
)

To apply the crossover on the whole population, we have to
1. Shuffle the population
1. Group parents in pairs
1. For each pair, pick a random number according to a crossover probability
   If no crossover for the current pair, copy the parents chromosomes into the children ones

In [ ]:
crossover_probability = 0.8

def population_crossover(parents: np.ndarray, crossover_probability: float, rng: np.random.Generator, display_crossovers: bool = False) -> np.ndarray:
    # shuffle the parents
    parents = parents[rng.permutation(parents.shape[0])]
    children = np.copy(parents)
    for i in np.arange(0,parents.shape[0],2): # two by two
        parent1 = parents[i,:]
        parent2 = parents[i+1,:]
        if rng.uniform(low=0.0, high=1.0) < crossover_probability:
            # generate a random crossover point
            crossover_point = rng.integers(low=0, high=6, size=1, dtype=int, endpoint=True)[0] 
            children[i,:], children[i+1,:] = individuals_crossover(
                parent1,
                parent2,
                crossover_point
            )
            if display_crossovers:
                display_crossover(
                    parent1,
                    parent2,
                    crossover_point,
                    children[i,:],
                    children[i+1,:]
                )
        else:
            # no crossover, leave parents chromosomes in children[i,:] and children[i+1,:]
            if display_crossovers:
                # like display_crossover() but no colors
                print(
                    f'parent1 : {chromosome2str(parent1)}\n'
                    f'parent2 : {chromosome2str(parent2)}\n'
                    f' child1 : {chromosome2str(children[i,:])}\n'
                    f' child2 : {chromosome2str(children[i+1,:])}\n'
                )
    return children

children = population_crossover(population,crossover_probability,rng,True)

# Mutation

At random, some gene the individuals are altered to express the other allele (bit flip)

In [ ]:
mutation_probability = 0.08

def population_mutation(population: np.ndarray, mutation_probability: float, rng: np.random.Generator, display_mutations: bool = False) -> np.ndarray:
    to_print = ''
    for idx in range(population.shape[0]):
        if display_mutations:
            to_print += chromosome2str(population[idx,:]) + ' -> '
        for gene in range(8):
            if rng.uniform(low=0.0, high=1.0) < mutation_probability:
                population[idx,gene] = int(not bool(population[idx,gene])) # binary complement
                if display_mutations:
                    to_print += f'[orange1]{population[idx,gene]}[/]'
            else:
                if display_mutations:
                    to_print += str(population[idx,gene])
        if display_mutations:
            to_print += '\n'
    if display_mutations:
        console.print(to_print)

population_mutation(children,mutation_probability,rng,True)

# Selection

It consist of replacing some of the parents (individuals of the last generation) with some children (individuals go through crossover and mutations), to obtain a new generation.

Several strategies are possible:
- Keeping the overall $N$ best individuals
- Replacing the $n$ worse parents by the $n$ best children
- Roulette wheel
- Tournament

Here we will implement the 2nd one.

In [ ]:
def indices_of_the_best(scores: np.ndarray, n: int) -> np.ndarray:
    original_indices = np.arange(scores.shape[0])
    sorting_indices = np.argsort(scores,0) # the indices sorting the scores
    sorted_original_indices = original_indices[sorting_indices[::-1]] # apply them on the original indices
    return sorted_original_indices[:n] # keep only the n^th first

def indices_of_the_worse(scores: np.ndarray, n: int) -> np.ndarray:
    original_indices = np.arange(scores.shape[0])
    sorting_indices = np.argsort(scores,0) # the indices sorting the scores
    sorted_original_indices = original_indices[sorting_indices[::1]] # apply them on the original indices
    return sorted_original_indices[:n] # keep only the n^th first

n = 10 # the 10 worse parents will be replaced by the 10 best children

def selection(parents: np.ndarray, children: np.ndarray, n, display_diff: bool = False) -> np.ndarray:
    new_population = np.copy(parents)

    # recompute the scores of the parents, because the crossover shuffled them
    parents_scores = evaluate_population(parents)

    # compute the scores of the children
    children_scores = evaluate_population(children)

    indices_of_worse_parents = indices_of_the_worse(parents_scores,n)
    indices_of_best_children = indices_of_the_best(children_scores,n)

    if display_diff:
        # based on display_scores()
        table = Table(title="Parents")
        table.add_column('Index')
        table.add_column('Chromosome')
        table.add_column('Score')
        for idx in range(0,parents.shape[0]):
            score_str = f'{parents_scores[idx,0]:0.2f}'
            table.add_row(
                str(idx),
                chromosome2str(parents[idx,:]),
                score_str if idx not in indices_of_worse_parents else ('[bright_red]'+score_str+'[/]')
            )
        console.print(table)
        table = Table(title="Children")
        table.add_column('Index')
        table.add_column('Chromosome')
        table.add_column('Score')
        for idx in range(0,children.shape[0]):
            score_str = f'{children_scores[idx,0]:0.2f}'
            table.add_row(
                str(idx),
                chromosome2str(children[idx,:]),
                score_str if idx not in indices_of_best_children else ('[bright_green]'+score_str+'[/]')
            )
        console.print(table)
    
    # actual replacement
    for i in range(n):
        # replacement in the population of one of the worse parent
        # by one of the best child
        new_population[indices_of_worse_parents[i],:] = children[indices_of_best_children[i],:]
    return new_population
    # TODO also update & return scores?

# replace `population` with the new population
population = selection(population,children,n,True)

Evaluate and plot the new generation

In [ ]:
generation = 1
scores = evaluate_population(population)
print('new population:')
display_scores(population,scores,console)
mean, max, std = compute_stats(scores)
print_generation_stats(mean,max,std)

# plot the new population (generation 1)
fig = plot_population(x,y,population)
fig.update_layout(title_text=f'Generation {generation}')
fig.show()

if EXPORT_FIGURES:
    fig.write_image(f'generation_{generation}.png')

---

In [ ]:
proba_croisement = 0.8
proba_mutation = 0.08
max_generations = 100
nb_individus_remplaces = 5
afficher_etapes = True
afficher_evolution_du_score = True
scores_moyens = np.zeros((1,max_generations+1))
meilleurs_scores = np.zeros((1,max_generations+1))
boucle_optimisation(population,proba_croisement,proba_mutation,max_generations,nb_individus_remplaces,afficher_etapes,afficher_evolution_du_score,scores_moyens,meilleurs_scores)

Then to obain a GIF of the evolving population, with [Gifski](https://github.com/ImageOptim/gifski/) :
```bash
gifski -o anim.gif generation_*.png --fps 4
```